In [1]:
%load_ext autoreload
%autoreload 2
%cd /group/transreg/sathi/DeepDifE 

import pickle
import importlib
import esparto
import optuna
import numpy as np
import pandas as pd
from evoaug_tf import evoaug, augment
from src.diff_expression_model import get_model, get_siamese_model, post_hoc_conjoining, get_auroc
from skopt.utils import use_named_args

/storage/nas6/group/biocomp/projects/transreg/sathi/DeepDifE


2024-07-08 11:41:53.534300: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /software/shared/apps/x86_64/dependencies_rl9:/software/shared/apps/x86_64/git/2.13.1/lib64/
2024-07-08 11:41:53.534388: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-08 11:42:19.075136: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /software/shared/apps/x86_64/dependencies_rl9:/software/shared/apps/x86_64/git/2.13.1/lib64/
2024-07-08 11:42:19.075860: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynam

## Prepare the data

For now we start from this data pickle as I'm not aware how Helder did DE analysis and generated the labels

In [2]:
ppath = "/group/transreg/heopd/dlpipe/results/ath/aba/dlresults/predetermined_dataset/dataset_solid_chrome.pkl"
with open(ppath, 'rb') as f:
    data = pickle.load(f)

In [3]:
data.columns

Index(['Category', 'GeneFamily', 'seqs', 'ohs', 'rcohs', 'ohsDuo',
       'in_original_balanced', 'set', 'npshap-single', 'npshap-posthoc'],
      dtype='object')

In [4]:
data

,Category,GeneFamily,seqs,ohs,rcohs,ohsDuo,in_original_balanced,set,npshap-single,npshap-posthoc
geneID,,,,,,,,,,
AT4G27120,0,HOM04D000881,TAGAGAAGACAAGCGGTTATTTCGTAATTTCCCAGCGACTTTGAAA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [1...",True,train,"[[[[-0.00029412], [0.00011191], [0.00071743], ...","[[[[-0.00024759], [0.00013361], [0.00015087], ..."
AT4G19600,0,HOM04D000740,GTCAAGTAGTGAAATCAAGGTGTGAAGTAAGCTGAGGACAGATAAT...,"[[0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1,...","[[0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0,...","[[[0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1...",True,valid,"[[[[0.00049611], [0.00069235], [0.00377408], [...","[[[[0.00051932], [0.00032187], [0.00027891], [..."
AT3G60880,0,HOM04D003119,AGTTGATATTGAATGAAATCTTCATGTTTTTTGATAAATGATTATA...,"[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,...","[[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0...",True,valid,"[[[[-0.00030243], [-1.08283739e-05], [0.000359...","[[[[-0.00031496], [-8.34961975e-05], [0.000132..."
AT5G06960,0,HOM04D000319,CACTTGTCAGATTCTTCTTACCAAATCCATCAACAAATAAGCAAAT...,"[[0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,...","[[[0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0...",True,train,"[[[[0.00254294], [0.00639127], [0.00152307], [...","[[[[0.00208253], [0.0018626], [0.00023493], [0..."
AT1G14890,0,HOM04D000273,TTGATATAACAGATTCAACACTAAAAATGAGTAAAATCTAAAAAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1,...","[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [1,...","[[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1...",True,valid,"[[[[-5.23597009e-05], [0.00045373], [0.0003575...","[[[[0.00051831], [0.00050734], [4.30682958e-05..."
...,...,...,...,...,...,...,...,...,...,...
AT5G64230,1,HOM04D003278,AAGAAAGAAAAACCGTACATAAACACCCATCTGGTATACCATCGTC...,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,...","[[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1...",True,test,"[[[[-1.85048248e-05], [5.31432997e-05], [0.000...","[[[[0.00034249], [0.00039158], [1.82568183e-05..."
AT5G64780,1,HOM04D002552,TTTTAGAAAGAAGAAGAAGGATTATTGCCTTATTGGTGAAGGGAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,...","[[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0...",True,train,"[[[[9.53665585e-05], [0.00032812], [0.00046541...","[[[[-5.87131144e-05], [6.68894427e-05], [6.172..."
AT4G30470,1,HOM04D000082,TATGTACAGTCTCTACATTTTTTCAAATACATTTTTTTCTTTTTCA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [0...",True,test,"[[[[0.00061825], [6.60565856e-05], [5.89712399...","[[[[0.00044855], [6.3632986e-05], [0.00018469]..."


In [5]:
data["set"].value_counts()

train    1900
valid     257
test      241
Name: set, dtype: int64

To show how to subdivide the dataset into train-test split we only take the following columns

In [6]:
dataset = data.reset_index()
dataset = dataset[["geneID", "Category", "GeneFamily", "seqs"]]
dataset.rename(columns={"geneID":"GeneID", "Category":"Label", "seqs": "Sequence"}, inplace=True)

In [7]:
dataset

,GeneID,Label,GeneFamily,Sequence
0,AT4G27120,0,HOM04D000881,TAGAGAAGACAAGCGGTTATTTCGTAATTTCCCAGCGACTTTGAAA...
1,AT4G19600,0,HOM04D000740,GTCAAGTAGTGAAATCAAGGTGTGAAGTAAGCTGAGGACAGATAAT...
2,AT3G60880,0,HOM04D003119,AGTTGATATTGAATGAAATCTTCATGTTTTTTGATAAATGATTATA...
3,AT5G06960,0,HOM04D000319,CACTTGTCAGATTCTTCTTACCAAATCCATCAACAAATAAGCAAAT...
4,AT1G14890,0,HOM04D000273,TTGATATAACAGATTCAACACTAAAAATGAGTAAAATCTAAAAAAG...
...,...,...,...,...
2393,AT5G64230,1,HOM04D003278,AAGAAAGAAAAACCGTACATAAACACCCATCTGGTATACCATCGTC...
2394,AT5G64780,1,HOM04D002552,TTTTAGAAAGAAGAAGAAGGATTATTGCCTTATTGGTGAAGGGAAG...
2395,AT4G30470,1,HOM04D000082,TATGTACAGTCTCTACATTTTTTCAAATACATTTTTTTCTTTTTCA...
2396,AT3G51895,1,HOM04D000270,TGGTAAATAATTAAATATATAAGAACATTATTCTAAAGCGTTGAAT...


## Phylo augmentation 

In [9]:
phylo_augmentation_base_path = "/group/transreg/sathi/phyloAugData/promotors_helder/output_filtered/"
species_fasta_files = [
			"ath-an1.tss.fasta",
			"ath-c24.tss.fasta",
			"ath-cvi.tss.fasta",
			"ath-eri.tss.fasta",
			"ath-kyo.tss.fasta",
			"ath-ler.tss.fasta",
			"ath-sha.tss.fasta"
]

In [26]:
from Bio import SeqIO

sequence_list = []
gene_id_list = []
label_list = []
species_list = []
for species_fasta in species_fasta_files:
	fasta_path = f"{phylo_augmentation_base_path}{species_fasta}"
	fasta_sequences = SeqIO.parse(open(fasta_path),'fasta')
	print(f"File: {fasta_path} done!")
	for fasta in fasta_sequences:
		name, sequence = fasta.id, str(fasta.seq)
		
		# The label is stored inside the id name (e.g. >ATERI-4G42060_1), split these
		gene_id = name.split("_")[0]
		label = name.split("_")[1]
		species = species_fasta.split(".")[0]

		gene_id_list.append(gene_id)
		sequence_list.append(sequence)
		label_list.append(label)
		species_list.append(species)

File: /group/transreg/sathi/phyloAugData/promotors_helder/output_filtered/ath-an1.tss.fasta done!
File: /group/transreg/sathi/phyloAugData/promotors_helder/output_filtered/ath-c24.tss.fasta done!
File: /group/transreg/sathi/phyloAugData/promotors_helder/output_filtered/ath-cvi.tss.fasta done!
File: /group/transreg/sathi/phyloAugData/promotors_helder/output_filtered/ath-eri.tss.fasta done!
File: /group/transreg/sathi/phyloAugData/promotors_helder/output_filtered/ath-kyo.tss.fasta done!
File: /group/transreg/sathi/phyloAugData/promotors_helder/output_filtered/ath-ler.tss.fasta done!
File: /group/transreg/sathi/phyloAugData/promotors_helder/output_filtered/ath-sha.tss.fasta done!


In [27]:
phylo_aug_df = pd.DataFrame({"GeneID": gene_id_list, "Label": label_list, "Sequence": sequence_list, "Species": species_list})

In [28]:
phylo_aug_df

,GeneID,Label,Sequence,Species
0,ATAN-3G78750,1,TTCAGGAGATTTTTGTTATCTTTGGTGAAAATAGAGTCAAACTCGG...,ath-an1
1,ATAN-3G15180,0,CAGAGTCACATCGTACATGCCATTGGTTCCAACAGAGAGGATCTCA...,ath-an1
2,ATAN-3G66750,0,CTTGACTTAATCCTCAAAACCAAATCTAACCTTAACAAGGAAAGTC...,ath-an1
3,ATAN-3G40170,0,ATATAAAAAGTAGACTCAAAATGCCTTTTTATTATTATTTTCGGCA...,ath-an1
4,ATAN-5G66410,1,AAGCAAAAAAGCTATTATCGTCCCATTGTGATATGTTTGAATTTTT...,ath-an1
...,...,...,...,...
13591,ATSHA-1G32710,0,TCAACGAATAAGGAATTCAAAAAGGAGCTCAAAATCGAAAAAGATC...,ath-sha
13592,ATSHA-1G17160,1,AGTCTTGAAAAGTTGACCTAATTTAGATCAAATTAAGGTTAACTAC...,ath-sha
13593,ATSHA-2G55860,0,TTGTCTCTATTATCGAATCTTATACTAACATCTTAGTTAGTACATA...,ath-sha
13594,ATSHA-5G86510,0,AGATTTCCGTCGGTCTATTTATTTATTATCTTTAATTTTCAGTATC...,ath-sha


### Get gene family of the original ortholog

In [60]:
base_dir_ortholog = "/group/transreg/sathi/phyloAugData/promotors_helder/data/orthologs/"
ortholog_files = {
			"ath-aar":"ath_aar.txt",
			"ath-aly":"ath_aly.txt",
			"ath-an1":"ath_ath-an1.txt",
			"ath-c24":"ath_ath-c24.txt",
			"ath-cvi":"ath_ath-cvi.txt",
			"ath-eri":"ath_ath-eri.txt",
			"ath-kyo":"ath_ath-kyo.txt",
			"ath-ler":"ath_ath-ler.txt",
			"ath-sha":"ath_ath-sha.txt",
			"ath-ath":"ath_ath.txt",
			"ath-chi":"ath_chi.txt",
			"ath-cpa":"ath_cpa.txt",
			"ath-cru":"ath_cru.txt",
			"ath-esa":"ath_esa.txt",
			"ath-spa":"ath_spa.txt",
			"ath-tha":"ath_tha.txt"
			}

In [33]:
f"{base_dir_ortholog}{ortholog_files['ath_aar']}"

'/group/transreg/sathi/phyloAugData/promotors_helder/data/orthologs/ath_aar.txt'

In [41]:
# Create a column with, for now, empty gene family information
phylo_aug_df["GeneFamily"] = ""

In [42]:
phylo_aug_df

,GeneID,Label,Sequence,Species,GeneFamily,Ortholog
0,ATAN-3G78750,1,TTCAGGAGATTTTTGTTATCTTTGGTGAAAATAGAGTCAAACTCGG...,ath-an1,,
1,ATAN-3G15180,0,CAGAGTCACATCGTACATGCCATTGGTTCCAACAGAGAGGATCTCA...,ath-an1,,
2,ATAN-3G66750,0,CTTGACTTAATCCTCAAAACCAAATCTAACCTTAACAAGGAAAGTC...,ath-an1,,
3,ATAN-3G40170,0,ATATAAAAAGTAGACTCAAAATGCCTTTTTATTATTATTTTCGGCA...,ath-an1,,
4,ATAN-5G66410,1,AAGCAAAAAAGCTATTATCGTCCCATTGTGATATGTTTGAATTTTT...,ath-an1,,
...,...,...,...,...,...,...
13591,ATSHA-1G32710,0,TCAACGAATAAGGAATTCAAAAAGGAGCTCAAAATCGAAAAAGATC...,ath-sha,,
13592,ATSHA-1G17160,1,AGTCTTGAAAAGTTGACCTAATTTAGATCAAATTAAGGTTAACTAC...,ath-sha,,
13593,ATSHA-2G55860,0,TTGTCTCTATTATCGAATCTTATACTAACATCTTAGTTAGTACATA...,ath-sha,,
13594,ATSHA-5G86510,0,AGATTTCCGTCGGTCTATTTATTTATTATCTTTAATTTTCAGTATC...,ath-sha,,


In [96]:
lookup_table = {}
for orthologs in ortholog_files.values():
	with open(f"{base_dir_ortholog}{orthologs}", 'r') as file:
		for line in file:
			# Split each line by tab
			columns = line.strip().split('\t')
			if len(columns) >= 2:  # Ensure there are at least two columns
				key = columns[1]
				value = columns[0]
				lookup_table[key] = value

In [99]:
phylo_aug_df["Ortholog"] = phylo_aug_df["GeneID"].map(lookup_table)

In [113]:
phylo_aug_df

,GeneID,Label,Sequence,Species,Ortholog
0,ATAN-3G78750,1,TTCAGGAGATTTTTGTTATCTTTGGTGAAAATAGAGTCAAACTCGG...,ath-an1,AT3G60150
1,ATAN-3G15180,0,CAGAGTCACATCGTACATGCCATTGGTTCCAACAGAGAGGATCTCA...,ath-an1,AT3G05510
2,ATAN-3G66750,0,CTTGACTTAATCCTCAAAACCAAATCTAACCTTAACAAGGAAAGTC...,ath-an1,AT3G49100
3,ATAN-3G40170,0,ATATAAAAAGTAGACTCAAAATGCCTTTTTATTATTATTTTCGGCA...,ath-an1,AT3G27150
4,ATAN-5G66410,1,AAGCAAAAAAGCTATTATCGTCCCATTGTGATATGTTTGAATTTTT...,ath-an1,AT5G45310
...,...,...,...,...,...
13591,ATSHA-1G32710,0,TCAACGAATAAGGAATTCAAAAAGGAGCTCAAAATCGAAAAAGATC...,ath-sha,AT1G21200
13592,ATSHA-1G17160,1,AGTCTTGAAAAGTTGACCTAATTTAGATCAAATTAAGGTTAACTAC...,ath-sha,AT1G07430
13593,ATSHA-2G55860,0,TTGTCTCTATTATCGAATCTTATACTAACATCTTAGTTAGTACATA...,ath-sha,AT2G36740
13594,ATSHA-5G86510,0,AGATTTCCGTCGGTCTATTTATTTATTATCTTTAATTTTCAGTATC...,ath-sha,AT5G61970


In [114]:
phylo_aug_df = pd.merge(phylo_aug_df, dataset[["GeneID", "GeneFamily"]], left_on="Ortholog", right_on="GeneID", how="left")

In [116]:
phylo_aug_df.drop("GeneID_y", axis=1, inplace=True)
phylo_aug_df.rename(columns={"GeneID_x": "GeneID"}, inplace=True)
phylo_aug_df

,GeneID,Label,Sequence,Species,Ortholog,GeneFamily
0,ATAN-3G78750,1,TTCAGGAGATTTTTGTTATCTTTGGTGAAAATAGAGTCAAACTCGG...,ath-an1,AT3G60150,HOM04D006128
1,ATAN-3G15180,0,CAGAGTCACATCGTACATGCCATTGGTTCCAACAGAGAGGATCTCA...,ath-an1,AT3G05510,HOM04D002504
2,ATAN-3G66750,0,CTTGACTTAATCCTCAAAACCAAATCTAACCTTAACAAGGAAAGTC...,ath-an1,AT3G49100,HOM04D004899
3,ATAN-3G40170,0,ATATAAAAAGTAGACTCAAAATGCCTTTTTATTATTATTTTCGGCA...,ath-an1,AT3G27150,HOM04D000675
4,ATAN-5G66410,1,AAGCAAAAAAGCTATTATCGTCCCATTGTGATATGTTTGAATTTTT...,ath-an1,AT5G45310,HOM04D006599
...,...,...,...,...,...,...
13591,ATSHA-1G32710,0,TCAACGAATAAGGAATTCAAAAAGGAGCTCAAAATCGAAAAAGATC...,ath-sha,AT1G21200,HOM04D001324
13592,ATSHA-1G17160,1,AGTCTTGAAAAGTTGACCTAATTTAGATCAAATTAAGGTTAACTAC...,ath-sha,AT1G07430,HOM04D000397
13593,ATSHA-2G55860,0,TTGTCTCTATTATCGAATCTTATACTAACATCTTAGTTAGTACATA...,ath-sha,AT2G36740,HOM04D006542
13594,ATSHA-5G86510,0,AGATTTCCGTCGGTCTATTTATTTATTATCTTTAATTTTCAGTATC...,ath-sha,AT5G61970,HOM04D000550


In [ ]:
a=1

### Sample if needed to make sure the ratio augmented-real samples is not too big

In [21]:
pass

### Merge with real dataset

### One-hot-encode & reverse-complement

In [7]:
from src.prepare_dataset import one_hot_encode_series, reverse_complement_series, reverse_complement_sequence
dataset["One_hot_encoded"] = one_hot_encode_series(dataset["Sequence"])

In [8]:
dataset["RC_one_hot_encoded"] = reverse_complement_series(dataset["One_hot_encoded"])

In [9]:
dataset

,GeneID,Label,GeneFamily,Sequence,One_hot_encoded,RC_one_hot_encoded
0,AT4G27120,0,HOM04D000881,TAGAGAAGACAAGCGGTTATTTCGTAATTTCCCAGCGACTTTGAAA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,..."
1,AT4G19600,0,HOM04D000740,GTCAAGTAGTGAAATCAAGGTGTGAAGTAAGCTGAGGACAGATAAT...,"[[0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1,...","[[0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
2,AT3G60880,0,HOM04D003119,AGTTGATATTGAATGAAATCTTCATGTTTTTTGATAAATGATTATA...,"[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
3,AT5G06960,0,HOM04D000319,CACTTGTCAGATTCTTCTTACCAAATCCATCAACAAATAAGCAAAT...,"[[0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,..."
4,AT1G14890,0,HOM04D000273,TTGATATAACAGATTCAACACTAAAAATGAGTAAAATCTAAAAAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1,...","[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [1,..."
...,...,...,...,...,...,...
2393,AT5G64230,1,HOM04D003278,AAGAAAGAAAAACCGTACATAAACACCCATCTGGTATACCATCGTC...,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,..."
2394,AT5G64780,1,HOM04D002552,TTTTAGAAAGAAGAAGAAGGATTATTGCCTTATTGGTGAAGGGAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
2395,AT4G30470,1,HOM04D000082,TATGTACAGTCTCTACATTTTTTCAAATACATTTTTTTCTTTTTCA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,..."
2396,AT3G51895,1,HOM04D000270,TGGTAAATAATTAAATATATAAGAACATTATTCTAAAGCGTTGAAT...,"[[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1,..."


### Train-test split

In [10]:
from src.prepare_dataset import grouped_shuffle_split
train_df, validation_test_df = grouped_shuffle_split(dataset, dataset["GeneFamily"], 0.2)

In [11]:
validation_df, test_df  = grouped_shuffle_split(validation_test_df, validation_test_df["GeneFamily"], 0.5)

In [12]:
print(f"Length of training set: {train_df.shape[0]}")
print(f"Length of validation set: {validation_df.shape[0]}")
print(f"Length of test set: {test_df.shape[0]}")

Length of training set: 1900
Length of validation set: 257
Length of test set: 241


In [13]:
def get_input_and_labels(df):
	ohe_np = np.stack(df["One_hot_encoded"])
	rc_np = np.stack(df["RC_one_hot_encoded"])

	x = np.append(ohe_np, rc_np, axis=0)
	x = x.astype('float32')
	y = np.append(df["Label"], df["Label"])
	return x, y

In [14]:
x_train, y_train = get_input_and_labels(train_df)
x_validation, y_validation = get_input_and_labels(validation_df)

## Prepare model

As the model uses evo augmentation, a list of possible nucleotide operations needs to be given

In [15]:
augment_list = [
    augment.RandomRC(rc_prob=0.5),
    augment.RandomInsertionBatch(insert_min=0, insert_max=20),
    augment.RandomDeletion(delete_min=0, delete_max=30),
    augment.RandomTranslocationBatch(shift_min=0, shift_max=20),
    augment.RandomMutation(mutate_frac=0.05),
    augment.RandomNoise()
]

2024-07-08 09:35:55.174789: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /software/shared/apps/x86_64/dependencies_rl9:/software/shared/apps/x86_64/git/2.13.1/lib64/
2024-07-08 09:35:55.174873: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-08 09:35:55.174946: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cyclone4.psblocal): /proc/driver/nvidia/version does not exist


Get the shape of the input data

In [16]:
input_shape = train_df["One_hot_encoded"].iloc[0].shape

Initialize the model

In [17]:
model = get_model(input_shape=input_shape, perform_evoaug=True, augment_list=augment_list,learning_rate=0.001)

## Train the model

As the hyperparameter parameters are set, we will do one more run with all possible training data

In [18]:
x_full_train = np.append(x_train, x_validation, axis=0)
y_full_train = np.append(y_train, y_validation)

In [19]:
# early stopping callback
import tensorflow as tf

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
											patience=20,
											verbose=1,
											mode='min',
											restore_best_weights=True)
# reduce learning rate callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
												factor=0.1,
												patience=5,
												min_lr=1e-7,
												mode='min',
												verbose=1)
callbacks = [early_stopping_callback, reduce_lr]

In [20]:
history = model.fit(x_train,
					y_train,
					epochs=100,
					batch_size=100,
					validation_data=(x_validation, y_validation),
					callbacks=callbacks
					)

Epoch 1/100
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
38/38 [==============================] - 92s 1s/step - loss: 0.6940 - acc: 0.5045 - auROC: 0.5053 - auPRC: 0.4971 - true_positives: 1003.0000 - auroc: 0.5053 - val_loss: 0.6962 - val_acc: 0.4981 - val_auROC: 0.5586 - val_auPRC: 0.5428 - val_true_positives: 0.0000e+00 - val_auroc: 0.5586 - lr: 0.0010
Epoch 2/100
38/38 [==============================] - 40s 1s/step - loss: 0.6937 - acc: 0.5118 - auROC: 0.5081 - auPRC: 0.5119 - true_positives: 563.0000 - auroc: 0.5081 - val_loss: 0.6933 - val_acc: 0.5019 - val_auROC: 0.5693 - val_auPRC: 0.5591 - val_true_positives: 258.0000 - val_auroc: 0.5693 - lr: 0.0010
Epoch 3/100
38/38 [==============================] - 45s 1s/step - loss: 0.6933 - acc: 0.5153 - auROC: 0.5131 - auPRC: 0.5111 - true_positives: 650.0000 - auroc: 0.5131 - val_l

In [21]:
min(history.history["val_loss"])

0.5952149033546448

In [22]:
siamese_model = get_siamese_model(model.model)

x_test = np.stack(test_df["One_hot_encoded"])
x_test_rc = np.stack(test_df["RC_one_hot_encoded"])

y_test = test_df["Label"].to_numpy()

x_test = model._pad_end(x_test)
x_test_rc = model._pad_end(x_test_rc)

predictions_categories, predictions = post_hoc_conjoining(siamese_model, x_test, x_test_rc)

get_auroc(y_test, predictions)

8/8 [==============================] - 1s 56ms/step


0.8164934164934164

### Finetune evo aug

In [23]:
from tensorflow import keras
# set up fine-tuning
finetune_optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.finetune_mode(optimizer=finetune_optimizer)


# set up callbacks
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
											patience=10,
											verbose=1,
											mode='min',
											restore_best_weights=True)
# train model
model.fit(x_train, y_train,
                epochs=20,
                batch_size=100,
                shuffle=True,
                validation_data=(x_validation, y_validation),
                callbacks=[early_stopping_callback])

Epoch 1/20
38/38 [==============================] - 42s 1s/step - loss: 0.5584 - acc: 0.7108 - auROC: 0.7895 - auPRC: 0.7737 - true_positives: 1240.0000 - auroc: 0.7895 - val_loss: 0.5943 - val_acc: 0.6829 - val_auROC: 0.7452 - val_auPRC: 0.7493 - val_true_positives: 186.0000 - val_auroc: 0.7452
Epoch 2/20
38/38 [==============================] - 43s 1s/step - loss: 0.5700 - acc: 0.6992 - auROC: 0.7770 - auPRC: 0.7706 - true_positives: 1219.0000 - auroc: 0.7770 - val_loss: 0.5947 - val_acc: 0.6868 - val_auROC: 0.7450 - val_auPRC: 0.7507 - val_true_positives: 184.0000 - val_auroc: 0.7450
Epoch 3/20
38/38 [==============================] - 42s 1s/step - loss: 0.5627 - acc: 0.7071 - auROC: 0.7826 - auPRC: 0.7680 - true_positives: 1242.0000 - auroc: 0.7826 - val_loss: 0.5963 - val_acc: 0.6887 - val_auROC: 0.7417 - val_auPRC: 0.7482 - val_true_positives: 188.0000 - val_auroc: 0.7417
Epoch 4/20
38/38 [==============================] - 42s 1s/step - loss: 0.5567 - acc: 0.7042 - auROC: 0.7878 

## Test set

We perform post-hoc conjoing

In [24]:
siamese_model = get_siamese_model(model.model)

Prepare the test set data

In [25]:
x_test = np.stack(test_df["One_hot_encoded"])
x_test_rc = np.stack(test_df["RC_one_hot_encoded"])

In [26]:
y_test = test_df["Label"].to_numpy()

As we used evo aug in our model all the sequences which were trained on, were of length 620. We use the evoaug padding function to pad the test set to the same length

In [27]:
x_test = model._pad_end(x_test)
x_test_rc = model._pad_end(x_test_rc)

In [28]:
predictions_categories, predictions = post_hoc_conjoining(siamese_model, x_test, x_test_rc)

8/8 [==============================] - 5s 623ms/step


In [29]:
get_auroc(y_test, predictions)

0.8167013167013167

In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions_categories))

              precision    recall  f1-score   support

           0       0.71      0.74      0.72       111
           1       0.77      0.74      0.75       130

    accuracy                           0.74       241
   macro avg       0.74      0.74      0.74       241
weighted avg       0.74      0.74      0.74       241

